# STT Model Training on Kaggle (Fixed & Tested)

**Successfully tested: January 2026**

**What this notebook does:**
1. Installs required packages
2. Transcribes audio with Whisper
3. Fine-tunes Whisper with LoRA
4. Saves trained model

**GPU Required:** Settings → Accelerator → GPU P100 or T4

## Step 1: Check GPU

In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("❌ Enable GPU: Settings → Accelerator → GPU")

## Step 2: Install Packages

In [ ]:
!pip install -q transformers datasets accelerate peft
!pip install -q librosa soundfile jiwer tensorboard
!pip install -q openai-whisper bitsandbytes torchcodec
print("✅ Packages installed!")

## Step 3: Setup Paths

In [ ]:
import os

# ============================================
# UPDATE THIS TO YOUR DATASET NAME!
# ============================================
DATASET_PATH = "/kaggle/input/stt-training-data"
OUTPUT_PATH = "/kaggle/working"

if os.path.exists(DATASET_PATH):
    print(f"✅ Dataset found: {DATASET_PATH}")
    for item in os.listdir(DATASET_PATH):
        print(f"  - {item}")
else:
    print("❌ Dataset not found!")
    print("Available:")
    for item in os.listdir("/kaggle/input"):
        print(f"  - /kaggle/input/{item}")

In [ ]:
# Find .wav files location
print("🔍 Searching for .wav files...\n")

AUDIO_BASE_PATH = None
for root, dirs, files in os.walk(DATASET_PATH):
    wav_files = [f for f in files if f.endswith('.wav')]
    if wav_files:
        print(f"✅ Found {len(wav_files)} .wav files in:")
        print(f"   {root}")
        AUDIO_BASE_PATH = root
        break

if not AUDIO_BASE_PATH:
    print("❌ No .wav files found!")

## Step 4: Transcribe Audio

In [ ]:
import whisper
import json
from tqdm import tqdm

print("Loading Whisper medium model...")
transcription_model = whisper.load_model("medium")
print("✅ Model loaded!")

def fix_audio_path(original_path, dataset_path):
    """Convert Windows path to Kaggle path."""
    path = original_path.replace('\\', '/')
    
    if 'extracted/' in path:
        relative = path.split('extracted/')[-1]
        # Try nested (from ZIP upload)
        nested = f"{dataset_path}/extracted/extracted/{relative}"
        if os.path.exists(nested):
            return nested
        # Try regular
        regular = f"{dataset_path}/extracted/{relative}"
        if os.path.exists(regular):
            return regular
        return nested
    elif 'combined/' in path:
        relative = path.split('combined/')[-1]
        return f"{dataset_path}/combined/{relative}"
    return original_path

def transcribe_manifest(manifest_path, lang, output_path):
    """Transcribe audio files from manifest."""
    with open(manifest_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    print(f"Found {len(lines)} files")
    
    # Test path
    sample = json.loads(lines[0].strip())
    sample_path = sample.get('audio_filepath') or sample.get('audio_path')
    fixed = fix_audio_path(sample_path, DATASET_PATH)
    print(f"Path test: {os.path.exists(fixed)}")
    
    if not os.path.exists(fixed):
        print(f"❌ Path not found: {fixed}")
        return []
    
    results = []
    for line in tqdm(lines, desc=f"Transcribing {lang}"):
        entry = json.loads(line.strip())
        orig_path = entry.get('audio_filepath') or entry.get('audio_path')
        audio_path = fix_audio_path(orig_path, DATASET_PATH)
        
        if os.path.exists(audio_path):
            try:
                result = transcription_model.transcribe(audio_path, language=lang)
                entry['text'] = result['text'].strip()
                entry['audio_filepath'] = audio_path
                results.append(entry)
            except:
                pass
    
    with open(output_path, 'w', encoding='utf-8') as f:
        for entry in results:
            f.write(json.dumps(entry, ensure_ascii=False) + '\n')
    
    print(f"✅ Transcribed: {len(results)} files")
    return results

In [ ]:
# Transcribe Hindi
hi_manifest = f"{DATASET_PATH}/combined/hi_train.jsonl"
if os.path.exists(hi_manifest):
    print(f"Found: {hi_manifest}")
    hi_results = transcribe_manifest(hi_manifest, "hi", f"{OUTPUT_PATH}/hi_train_transcribed.jsonl")
else:
    print("Hindi manifest not found")

In [ ]:
# Transcribe English
en_manifest = f"{DATASET_PATH}/combined/en_train.jsonl"
if os.path.exists(en_manifest):
    print(f"Found: {en_manifest}")
    en_results = transcribe_manifest(en_manifest, "en", f"{OUTPUT_PATH}/en_train_transcribed.jsonl")
else:
    print("English manifest not found")

## Step 5: Process Data (Direct Method - Reliable)

In [ ]:
import json
import librosa
import torch
from transformers import WhisperProcessor
from tqdm import tqdm

print("Loading processor...")
processor = WhisperProcessor.from_pretrained("openai/whisper-small")

# Load transcribed data
train_data = []
for filepath in [f"{OUTPUT_PATH}/hi_train_transcribed.jsonl", f"{OUTPUT_PATH}/en_train_transcribed.jsonl"]:
    if os.path.exists(filepath):
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                entry = json.loads(line.strip())
                if entry.get('text') and entry.get('audio_filepath'):
                    train_data.append(entry)

print(f"Loaded {len(train_data)} samples")

In [ ]:
# Process audio files directly
MAX_LABEL_LENGTH = 440

processed_data = []
errors = 0
skipped = 0

for entry in tqdm(train_data, desc="Processing"):
    audio_path = entry['audio_filepath']
    
    if not os.path.exists(audio_path):
        errors += 1
        continue
    
    try:
        audio, sr = librosa.load(audio_path, sr=16000)
        input_features = processor(audio, sampling_rate=16000, return_tensors="pt").input_features[0]
        labels = processor.tokenizer(entry['text']).input_ids
        
        if len(labels) > MAX_LABEL_LENGTH:
            skipped += 1
            continue
        
        processed_data.append({'input_features': input_features.numpy(), 'labels': labels})
    except:
        errors += 1

print(f"\n✅ Processed: {len(processed_data)}")
print(f"❌ Errors: {errors}")
print(f"⏭️ Skipped (long): {skipped}")

In [ ]:
from datasets import Dataset
processed_dataset = Dataset.from_list(processed_data)
print(f"✅ Dataset: {len(processed_dataset)} samples")

## Step 6: Setup Model with LoRA

In [ ]:
from transformers import WhisperForConditionalGeneration
from peft import LoraConfig, get_peft_model

print("Loading Whisper small...")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
print("✅ Model ready!")

## Step 7: Create Trainer

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": f["input_features"]} for f in features]
        label_features = [{"input_ids": f["labels"]} for f in features]

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"{OUTPUT_PATH}/whisper-finetuned",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=1e-4,
    warmup_steps=50,
    num_train_epochs=3,
    fp16=True,
    logging_steps=25,
    save_steps=500,
    save_total_limit=2,
    predict_with_generate=True,
    generation_max_length=225,
    report_to=["tensorboard"],
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=processed_dataset,
    data_collator=data_collator,
    processing_class=processor.feature_extractor,
)

print("✅ Trainer ready!")

## Step 8: Train!

In [ ]:
print("🚀 Starting training...")
trainer.train()
print("✅ Training complete!")

## Step 9: Save Model

In [ ]:
model_path = f"{OUTPUT_PATH}/whisper-stt-finetuned"
model.save_pretrained(model_path)
processor.save_pretrained(model_path)
print(f"✅ Saved to: {model_path}")

## Step 10: Test Model

In [ ]:
def transcribe(audio_path):
    audio, sr = librosa.load(audio_path, sr=16000)
    inputs = processor(audio, sampling_rate=16000, return_tensors="pt")
    with torch.no_grad():
        ids = model.generate(inputs.input_features.to(model.device), max_length=225)
    return processor.batch_decode(ids, skip_special_tokens=True)[0]

if train_data:
    test = train_data[0]
    print(f"Original: {test['text']}")
    print(f"Model: {transcribe(test['audio_filepath'])}")

## Step 11: Download

In [ ]:
import shutil
shutil.make_archive(f"{OUTPUT_PATH}/whisper-stt-model", 'zip', model_path)
print("✅ Download from Output panel →")

---

## How to Use Later

```python
from peft import PeftModel
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import librosa, torch

base = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model = PeftModel.from_pretrained(base, "./whisper-stt-finetuned")
processor = WhisperProcessor.from_pretrained("./whisper-stt-finetuned")

audio, sr = librosa.load("audio.wav", sr=16000)
inputs = processor(audio, sampling_rate=16000, return_tensors="pt")
ids = model.generate(inputs.input_features, max_length=225)
print(processor.batch_decode(ids, skip_special_tokens=True)[0])
```